In [1]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader,random_split
from torchvision import transforms, datasets, models
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch.nn.functional as F
import torch.nn.utils.prune as prune

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

mean = [0.4914, 0.4822, 0.4465] 
std = [0.2470, 0.2435, 0.2616] 
batch_size = 40
n_epochs = 100

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize all images to 224x224
    # Random augmentations
    # Randomly rotate images by 40 degrees
    transforms.RandomRotation(40),
    transforms.RandomHorizontalFlip(),  # Random horizontal flip
    transforms.ColorJitter(brightness=0.15, contrast=0.15, saturation=0.15, hue=0.05),  # Random color jitter
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize(mean=mean, std=std)  # Normalize with mean and std
])

path='train'
all_train = datasets.ImageFolder(root = path, transform = train_transform)
train_size = int(0.9 * len(all_train))
validation_size = len(all_train) - train_size
train_dataset, validation_dataset = random_split(all_train , [train_size, validation_size])
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=3
)
val_loader = DataLoader(
    validation_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=3
)

In [3]:
image_size = (3, 224, 224)  # Example: 3 channels, 32x32 pixels
num_classes = 100

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from math import ceil

# Inverted Residual Block with Squeeze-and-Excitation
class MBConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, expand_ratio, stride, kernel_size, reduction_ratio=4):
        super(MBConvBlock, self).__init__()
        self.stride = stride
        self.expand_ratio = expand_ratio
        hidden_dim = in_channels * expand_ratio
        
        # Expansion phase
        if expand_ratio != 1:
            self.expand_conv = nn.Conv2d(in_channels, hidden_dim, kernel_size=1, bias=False)
            self.bn0 = nn.BatchNorm2d(hidden_dim)
        else:
            self.expand_conv = None
        
        # Depthwise convolution
        self.depthwise_conv = nn.Conv2d(hidden_dim if expand_ratio != 1 else in_channels, hidden_dim, 
                                        kernel_size=kernel_size, stride=stride, 
                                        padding=kernel_size // 2, groups=hidden_dim, bias=False)
        self.bn1 = nn.BatchNorm2d(hidden_dim)
        
        # Squeeze and Excitation block
        self.se_avgpool = nn.AdaptiveAvgPool2d(1)
        self.se_fc1 = nn.Conv2d(hidden_dim, hidden_dim // reduction_ratio, kernel_size=1)
        self.se_fc2 = nn.Conv2d(hidden_dim // reduction_ratio, hidden_dim, kernel_size=1)
        
        # Output phase
        self.project_conv = nn.Conv2d(hidden_dim, out_channels, kernel_size=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        self.use_residual = (in_channels == out_channels and stride == 1)
    
    def forward(self, x):
        identity = x
        
        if self.expand_conv:
            out = F.relu6(self.bn0(self.expand_conv(x)))
        else:
            out = x
        
        # Depthwise convolution
        out = F.relu6(self.bn1(self.depthwise_conv(out)))
        
        # Squeeze and Excitation
        se = self.se_avgpool(out)
        se = F.relu(self.se_fc1(se))
        se = torch.sigmoid(self.se_fc2(se))
        out = out * se
        
        # Output
        out = self.bn2(self.project_conv(out))
        
        if self.use_residual:
            out = out + identity
        
        return out

# EfficientNet Main Architecture
class EfficientNet(nn.Module):
    def __init__(self, width_coefficient, depth_coefficient, dropout_rate=0.2, num_classes=100):
        super(EfficientNet, self).__init__()
        
        # Base settings for EfficientNet-B0 with reduced coefficients
        base_channels = 16  # Reduced base channels
        base_layers = [
            # (expand_ratio, out_channels, num_blocks, stride, kernel_size)
            (1, 16, 1, 1, 3),   # Stage 1
            (6, 24, 2, 2, 3),   # Stage 2
            (6, 40, 2, 2, 5),   # Stage 3
            (6, 80, 3, 2, 3),   # Stage 4
            (6, 112, 3, 1, 5),  # Stage 5
            (6, 192, 4, 2, 5),  # Stage 6
            (6, 320, 1, 1, 3)   # Stage 7
        ]
        
        # Stem
        out_channels = ceil(base_channels * width_coefficient)
        self.stem_conv = nn.Conv2d(3, out_channels, kernel_size=3, stride=2, padding=1, groups=1,bias=False)
        self.stem_bn = nn.BatchNorm2d(out_channels)
        
        # Build blocks
        self.blocks = nn.ModuleList([])
        in_channels = out_channels
        for expand_ratio, out_channels, num_blocks, stride, kernel_size in base_layers:
            out_channels = ceil(out_channels * width_coefficient)
            num_blocks = ceil(num_blocks * depth_coefficient)
            for i in range(num_blocks):
                block_stride = stride if i == 0 else 1
                self.blocks.append(MBConvBlock(in_channels, out_channels, expand_ratio, block_stride, kernel_size))
                in_channels = out_channels
        
        # Head
        final_channels = ceil(1024 * width_coefficient)  # Reduced head channels
        self.head_conv = nn.Conv2d(in_channels, final_channels, kernel_size=1, bias=False)
        self.head_bn = nn.BatchNorm2d(final_channels)
        
        # Pooling and classification
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(final_channels, num_classes)
    
    def forward(self, x):
        # Stem
        x = F.relu6(self.stem_bn(self.stem_conv(x)))
        
        # Blocks
        for block in self.blocks:
            x = block(x)
        
        # Head
        x = F.relu6(self.head_bn(self.head_conv(x)))
        
        # Pooling and classification
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = self.fc(x)
        
        return x

def efficientnet_b0(num_classes=100):
    # Reduced width and depth coefficients to reduce parameters
    return EfficientNet(width_coefficient=0.8, depth_coefficient=0.85, num_classes=num_classes)

In [5]:
model = efficientnet_b0(num_classes=100).to(device)
prune.l1_unstructured(model.fc, name='weight', amount=0.4)
total_params = sum(p.numel() for p in model.parameters())
print("# parameters:", total_params)

# parameters: 4661839


In [6]:

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=1e-4, amsgrad=False)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=n_epochs, eta_min=1e-6)

# Early stopping class
class EarlyStopper:
    def __init__(self, patience=7, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = np.inf

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

# Train function
def train(model, train_loader, optimizer, loss_fn):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)
        
    epoch_loss = running_loss / len(train_loader.dataset)
    accuracy = correct / total
    return epoch_loss, accuracy

# Validation function
@torch.no_grad()
def validate(model, val_loader, loss_fn):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    for images, labels in tqdm(val_loader):
        images, labels = images.to(device), labels.to(device)
        
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        val_loss += loss.item() * images.size(0)
        
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)
        
    avg_loss = val_loss / len(val_loader.dataset)
    accuracy = correct / total
    return avg_loss, accuracy

# Training loop
train_loss_list = []
valid_loss_list = []
early_stopper = EarlyStopper(patience=7)

best_val_acc = 0.0

for epoch in range(n_epochs):
    train_loss, train_acc = train(model, train_loader, optimizer, loss_fn)
    val_loss, val_acc = validate(model, val_loader, loss_fn)
    
    train_loss_list.append(train_loss)
    valid_loss_list.append(val_loss)
    
    print(f"Epoch {epoch+1}/{n_epochs}:")
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
    
    # Step the learning rate scheduler
    scheduler.step()
    
    # Early stopping check
    if early_stopper.early_stop(val_loss):
        print("Early stopping")
        break
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_model.pth")

# Save the final model
torch.save(model.state_dict(), "final_model.pth")

  0%|          | 0/315 [00:00<?, ?it/s]

100%|██████████| 35/35 [00:15<00:00,  2.23it/s]


Epoch 1/100:
Train Loss: 4.1132, Train Acc: 0.0522
Val Loss: 3.7296, Val Acc: 0.0729


100%|██████████| 35/35 [00:11<00:00,  3.09it/s]


Epoch 2/100:
Train Loss: 3.5844, Train Acc: 0.1109
Val Loss: 3.4589, Val Acc: 0.1350


...

Epoch 73/100:
Train Loss: 0.1492, Train Acc: 0.9593
Val Loss: 0.9574, Val Acc: 0.7664


100%|██████████| 35/35 [00:18<00:00,  1.88it/s]


Epoch 74/100:
Train Loss: 0.1359, Train Acc: 0.9612
Val Loss: 0.9433, Val Acc: 0.7750


100%|██████████| 35/35 [00:09<00:00,  3.51it/s]

Epoch 75/100:
Train Loss: 0.1345, Train Acc: 0.9635
Val Loss: 1.0055, Val Acc: 0.7643
Early stopping
